![jupyter](environment.png)

Среда: кеточный мир. Агент может перемещаться из любой клетки влево или вправо.При перемещении агента из клетки 1 влево, агент остается в состоянии (клетке) 1.  Терминальное состояние клетка с номером 5.
Задача: построить оптимальную стратегию.
Цель: достигнуть терминальное состояние за минимальное количество шагов.

# Метод итераций по полезностям

См. лекция 11, слайд 22
Задаем reward каждого состояния, включая терминальное. 
Стратегия строится по полезностям. Агент выбирает действие, в котором поезность состояния выше.

In [1]:
import numpy as np
r = np.array([-1.,-1.,-1.,-1.,10])

Полезности состояний (задаем случайным образом)

In [11]:
 np.random.seed(0)
V = np.random.rand(5)
print(V)

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]


Матрица вероятностей перехода (при выборе равномерной стратегии, т.е. агент равновероятно передмещается влево и вправо)

In [5]:
P = np.array([[0.5, 0.5, 0., 0., 0.],[0.5, 0.,0.5, 0., 0.],[0., 0.5, 0., 0.5, 0.],[0., 0., 0.5, 0., 0.5],[0.,0.,0.,0.,1.]])
print(P)

[[0.5 0.5 0.  0.  0. ]
 [0.5 0.  0.5 0.  0. ]
 [0.  0.5 0.  0.5 0. ]
 [0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  1. ]]


In [12]:
gamma = 0.9
V = r+gamma*P@V
print(V)

[-0.43119871 -0.4817904  -0.43296735 -0.53811182 10.38128932]


In [13]:
V = r+gamma*P@V
print(V)

[-1.4108451  -1.38887473 -1.458956    3.47674488 19.34316039]


In [14]:
V = r+gamma*P@V
print(V)

[-2.25987392 -2.2914105  -0.06045843  7.04789197 27.40884435]


In [15]:
V = r+gamma*P@V
print(V)


[-3.04807799 -2.04414956  1.14041667 11.30677366 34.66795991]


4-я итерация привела к оптимальной стратегии

# Метод Монте Карло (с первым посещением)


См. Лекция 12, сайд 4
Стратегия агента - равномерная (т.е. вероятности перехода вправо и влево равны)
Поиск оптимальной стратегии основан на поезности.
gamma = 1

In [124]:
import numpy as np
np.random.seed(0)
V = np.random.rand(5)


In [125]:
# r = np.array([-1.,-1.,-1.,-1.,10]) 
def random_transition(state):
    t = np.random.randint(0,2)
    t = t*2-1
    state = state + t
    if state == 0:
        state = 1
    if state == 5:
        reward = 10
    else:
        reward = -1
    return state, reward

In [126]:
N = 200 # Количество итераций для Монте Карло
for j in range(N):
    for i in range(4):
        s = i+1
        REWARD = 0
        while s!=5:
            s,r = random_transition(s)
            REWARD +=r
        V[i] += REWARD 

In [127]:
V[:4] = V[:4]/N
V[4] = 10
print(V)


[-7.15725593 -5.19142405 -2.79198618  3.69272442 10.        ]


# Метод Временных различий 

См. Лекция 12, сайд 9 - 10
Стратегия агента - равномерная (т.е. вероятности перехода вправо и влево равны)

In [128]:
import numpy as np
np.random.seed(0)
V = np.random.rand(5)

In [129]:
N = 1000 # количество итераций
s = 1
a = 0.1
gamma = 0.9
for i in range(N):
    for j in range(4):
        s_next,r = random_transition(s)
    V[s-1] = V[s-1] + a*(r+gamma*V[s_next - 1]-V[s-1])
    if s_next == 5:
        s_next = 1
    s = s_next
 
    

In [131]:
V[4] = 10
print(V)


[-4.56054865 -3.45117327 -0.36684034  4.63766004 10.        ]


# Q learning

См. Лекция 12, сайд 33

В Q learning мы имеем дело с полезностью действия. Стратегия агента выбрать действие с максимальной стоимостью

In [137]:
Q = np.random.rand(2,4)
R = -np.ones((2,4))
R[1,3] = 10
print(R)

[[-1. -1. -1. -1.]
 [-1. -1. -1. 10.]]


In [158]:
def get_state(state):
    t = np.random.randint(0,2)
    t = t*2-1
    state = state + t
    if state == 0:
        state = 1
    return state

In [161]:
N = 1000 # количество итераций
s = 1
a = 0.1
gamma = 0.9
for j in range(N):
    for i in range(4):
        s = i+1
        s_next = get_state(s)
        r = -1   
        if s_next>s:
            idx = 1
            if s_next == 4:
                r == 10
        else:
            idx = 0
        if i < 3 or idx == 0:
            Q[idx,s-1] = Q[idx,s-1] + a*(r+gamma*np.max(Q[:,s_next-1])-Q[idx,s-1])
        else:
            Q[idx,s-1] = Q[idx,s-1] + a*(r+gamma*10-Q[idx,s-1])

In [162]:
print(Q)

[[1.8098 1.8098 3.122  4.58  ]
 [3.122  4.58   6.2    8.    ]]
